In [1]:
# reference : https://github.com/yaleCat/Grad-CAM-pytorch

import sys
sys.path.append('/')

import cv2
import numpy as np
from torchvision import models
from efficientnet_pytorch import EfficientNet
from grad_cam import GradCam,GuidedBackpropReLUModel,show_cam_on_image,show_gbs,preprocess_image
import os
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import json

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(42)
if device == 'cuda':
  torch.cuda.manual_seed_all(42)

In [3]:
valid_path = '/opt/ml/dataset/valid/JPEGImages'
file_list = os.listdir(valid_path)
image_list = []

for file in file_list:
    if '.jpg' in file:
        image_list.append(file)

In [4]:
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=5)

Loaded pretrained weights for efficientnet-b0


In [6]:
model_path = './epoch30/remove_background/model.pth'
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint, strict=False)

<All keys matched successfully>

In [7]:
num_blocks = len(model._blocks)
print(f'num of blocks : {num_blocks}')

num of blocks : 16


In [ ]:
from PIL import Image
from grad_cam import GradCam,GuidedBackpropReLUModel,show_cam_on_image,show_gbs,preprocess_image

idx = 300

# 마지막 레이어를 보도록 (원하는 레이어가 있으면 번호를 적어주세요. ex) ['19', '32']
target_layer_names = [str(num_blocks - 1)]

img = cv2.imread('./my_face.PNG', 1)
img_size = (256,256)
img = np.float32(cv2.resize(img, img_size))/255
inputs = preprocess_image(img)
grad_cam = GradCam(
    model=model,
    blob_name = '_blocks',
    target_layer_names=target_layer_names,
    use_cuda=True,
    img_size=img_size)
    
# 예측값 비교
out = model(inputs.to(device))
pred = torch.argmax(out, dim=-1)[0]
print(inputs.shape)
key = 'oil' # 항목
json_path = os.path.join(valid_path, image_list[idx].replace('jpg', 'json'))
with open(json_path, "r") as json_file:   # json 파일에 접근하여 json 파일을 불러온다.
    img_json = json.load(json_file)             
    
label = img_json[key]

print(f'predict: {pred}, gt: {label}')

# If None, returns the map for the highest scoring category.
# Otherwise, targets the requested index.
target_index = pred
mask_dic = grad_cam(inputs, target_index)
fig, axes = plt.subplots(nrows=1, ncols=len(target_layer_names) + 1, figsize=(10*(len(target_layer_names) + 1), 10))
axes[0].imshow(img[:, :, ::-1])
axes[0].axis('off')

for i, (name, mask) in enumerate(mask_dic.items()):
    heatmap = cv2.applyColorMap(np.uint8(255 * (1-mask)), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    heatmap[:,:,2] = 0
    cam = heatmap*0.3 + np.float32(img[:,:,::-1])
    cam = cam / np.max(cam)

    axes[i + 1].imshow(cam)
    axes[i + 1].axis('off')

plt.show()